## Notebook purpose:

See if we can use the multiprocessing package to run a timeconsuming function in parallel

In [1]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [2]:
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script> so much code! click <a href="javascript:code_toggle()">here</a>.''')

In [3]:
import matplotlib.pyplot as plt
import netCDF4 as nc
import numpy as np
import scipy as sp
import pandas as pd
import seawater
import datetime as dt
""
from salishsea_tools import (
    nc_tools,
    viz_tools,
    geo_tools,
    tidetools
)

import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import matplotlib.patches as patches
plt.style.use('seaborn-whitegrid')
import netCDF4 as nc
import pickle
import cmocean as cm
import glob
import sys
sys.path.append('/data/tjarniko/mocsy')
import mocsy

#from matplotlib import reload
import arrow
import gsw
import time

%matplotlib inline

plt.rcParams.update({'font.size': 20,
                     'xtick.labelsize' : 20,
                     'ytick.labelsize' : 20})

from multiprocessing import Process
import time
import BC_helper_methods as hm


In [4]:
def calc_preind_co2_AOU_method(arrowdate, obs_year, target_year, scen):
    
    
    ## using the AOU C* method to calculate DIC intrusions. takes ~20 minutes per day of boundary conditions. 
    import numpy as np
    import netCDF4 as nc
    import gsw
    
    test_LO = hm.load_nc(arrowdate)
    
    tdate = arrowdate
    yy = tdate.format('YYYY')
    mm = tdate.format('MM')
    dd = tdate.format('DD')
    ymd = f'y{yy}m{mm}d{dd}'
    #open dataset & retrieve relevant variables, calculate potential density

    zlevels = (test_LO['deptht'][:])
    sal = test_LO['vosaline'][0,:,0,:]
    temp = test_LO['votemper'][0,:,0,:]
    sigma0 = gsw.sigma0(sal,temp)
    DIC = test_LO['DIC'][0,:,0,:]
    TA = test_LO['TA'][0,:,0,:]
    O2 = test_LO['OXY'][0,:,0,:]
    depth_this = np.zeros_like(TA)
    zeros = np.zeros_like(TA)
    #depth_this - array of depths of same shape as DIC
    for i in range(0,950):
        depth_this[:,i] = zlevels
 
 ### GET AGE AND WATERMASS WITNESSED CO2   
    #calculate pycnal's last surfacing, according to exp function
    #found using cfc ages
    params0 = 0.1301889490932413
    params1 = 3.8509914822057825
    params2 = 8.301166081413104 #change to 2015 since model year is 2015


    water_age = (params0 *np.exp(-params1*(25.15-sigma0))+params2)
    # year_watermass_at_surface = int(targetyear - age)
    # watermass_witnessed_co2_obs = int(hm.co2_from_year(scen,year_watermass_at_surface))

    #get witnessed co2 both of the present day water parcel and the target year water parcel
    obs_year_ar = np.zeros_like(water_age)
    obs_year_ar[:] = obs_year
    
    target_year_ar = np.zeros_like(water_age)
    target_year_ar[:] = target_year
    
    year_watermass_at_surface = (obs_year_ar - water_age).astype(int)
    watermass_witnessed_co2_obs = hm.co2_from_year(scen,year_watermass_at_surface)
    print(np.min(watermass_witnessed_co2_obs))
    print(np.shape(watermass_witnessed_co2_obs))
    watermass_witnessed_co2_target = \
    hm.co2_from_year(scen,year_watermass_at_surface+(target_year_ar-obs_year_ar))

###GET AOU
#(1) estimate AOU on 26 (assoc with water parcel with DIC_{w,2019,26,jdf})
# = f(O2_{w,2019,26,jdf},S_{w,2019,26,jdf},T_{w,2019,26,jdf}, P_{w,2019,26,jdf})
#(P is there to determine T when last at surface - I'll call it preT next)
    AOU_stoich = hm.get_AOU_stoich(sal,temp,O2,sigma0,water_age)

 ### GET PREFORMED DIC        
    obs_preformed_dic = DIC - AOU_stoich

#### get preformed pco2 and target year preformed pco2
    pHr, OmAr, pco2r = hm.oned_moxy(sal, temp, obs_preformed_dic, TA, 1, np.zeros_like(sal))
    obsyear_pref_pco2 = pco2r
    diseqPCO2 = obsyear_pref_pco2 - watermass_witnessed_co2_obs
    targetyear_pref_pco2 = watermass_witnessed_co2_target + diseqPCO2

    print('calculating target year preformed DIC')    
    target_preformed_dic = np.zeros_like(DIC)
    target_preformed_dic_r = np.ravel(target_preformed_dic)
    targetyear_pref_pco2_r = np.ravel(targetyear_pref_pco2)
    depth_r = np.ravel(depth_this)
    sal_r = np.ravel(sal)
    temp_r = np.ravel(temp)
    TA_r = np.ravel(TA)
    zeros_r = np.zeros_like(TA_r)
 
    start = time.time()
    
    for i in range(0,len(TA_r)):
        if i%950 == 0:
            print(i)
        t_dic = hm.find_DIC_corresp_to_pco2(sal_r[i], temp_r[i], targetyear_pref_pco2_r[i], TA_r[i], 1, 0)
        target_preformed_dic_r[i] = t_dic
    target_preformed_dic = target_preformed_dic_r.reshape(40,950)
    
    print('seconds taken at the hard part')
    print(time.time()-start)
    
    deltaDIC = obs_preformed_dic - target_preformed_dic
    
    print('max deltaDIC: '+str(np.max(deltaDIC)) + ', min deltaDIC: '+ str(np.min(deltaDIC)))

    final_target_DIC = DIC - deltaDIC

#     target_year_ar = np.zeros_like(final_target_DIC)
#     target_year_ar[:] = target_year
    
    f = nc.Dataset(f'./JdF_future_DIC/TEST2_LO_targetyear_{target_year}_scenario_{scen}_{ymd}_DIC.nc','w', format='NETCDF4') #'w' stands for write
    g = f.createGroup('preindustrial_DIC')
    g.createDimension('xval', 950)
    g.createDimension('depth', 40)
    g.createDimension('single', 1)
    
    ts = g.createVariable('sigma0','f4',('depth','xval'))
    ts[:] = sigma0

    ts2 = g.createVariable('water_age','f4',('depth','xval'))
    ts2[:] = water_age

    ts2a = g.createVariable('target_year','f4',('single'))
    ts2a[:] = target_year  

#     ts3 = g.createVariable('watermass_witnessed_co2_obs','f4',('depth','xval'))
#     ts3[:] = watermass_witnessed_co2_obs

#     ts3a = g.createVariable('watermass_witnessed_co2_target','f4',('depth','xval'))
#     ts3a[:] = watermass_witnessed_co2_target
    
    ts4 = g.createVariable('AOU_stoich','f4',('depth','xval'))
    ts4[:] = AOU_stoich
    
    ts5 = g.createVariable('obsyear_pref_pco2','f4',('depth','xval'))
    ts5[:] = obsyear_pref_pco2
    ts5a = g.createVariable('targetyear_pref_pco2','f4',('depth','xval'))
    ts5a[:] = targetyear_pref_pco2
    
    ts5b = g.createVariable('obsyear_pref_dic','f4',('depth','xval'))
    ts5b[:] = obs_preformed_dic
    ts5c = g.createVariable('targetyear_pref_dic','f4',('depth','xval'))
    ts5c[:] = target_preformed_dic
    
    ts6 = g.createVariable('final_target_DIC','f4',('depth','xval'))
    ts6[:] = final_target_DIC

    f.close()



In [5]:
# import time
# import BC_helper_methods as hm
# import importlib
# importlib.reload(hm)


# start = '2015-01-01'
# start_run = arrow.get(start)

# start = time.time()
# calc_preind_co2_AOU_method(start_run, 2015, 2050, '2_4pt5')
# print('seconds taken')
# print(time.time()-start)

In [6]:
from multiprocessing import Process

start ='2016-01-01'
end ='2016-12-31'

start_run = arrow.get(start)
end_run = arrow.get(end)

arrow_array = []

for r in arrow.Arrow.span_range('day', start_run, end_run):
    arrow_array.append(r)
    
    
def func1():
  print('func1: starting')
  for i in range(0,60):
        calc_preind_co2_AOU_method(arrow_array[i][0], 2015, 2050, '2_4pt5')

def func2():
  print('func2: starting')
  for i in range(60,120):
        calc_preind_co2_AOU_method(arrow_array[i][0], 2015, 2050, '2_4pt5')
      
def func3():
  print('func3: starting')
  for i in range(120,180):
        calc_preind_co2_AOU_method(arrow_array[i][0], 2015, 2050, '2_4pt5')
        
def func4():
  print('func4: starting')
  for i in range(180,240):
        calc_preind_co2_AOU_method(arrow_array[i][0], 2015, 2050, '2_4pt5')
        
def func5():
  print('func5: starting')
  for i in range(240,300):
        calc_preind_co2_AOU_method(arrow_array[i][0], 2015, 2050, '2_4pt5')
        
def func6():
  print('func36: starting')
  for i in range(300,366):
        calc_preind_co2_AOU_method(arrow_array[i][0], 2015, 2050, '2_4pt5')
        
if __name__ == '__main__':
  p1 = Process(target=func1)
  p1.start()
  p2 = Process(target=func2)
  p2.start()
  p3 = Process(target=func3)
  p3.start()
  p4 = Process(target=func4)
  p4.start()
  p5 = Process(target=func5)
  p5.start()
  p6 = Process(target=func6)
  p6.start()                
    
  p1.join()
  p2.join()
  p3.join()
  p4.join()
  p5.join()
  p6.join()

func1: starting
2016-01-01T00:00:00+00:00
func2: starting
2016-03-01T00:00:00+00:00
func3: starting
2016-04-30T00:00:00+00:00
func4: starting
2016-06-29T00:00:00+00:00
func5: starting
2016-08-28T00:00:00+00:00
func36: starting
2016-10-27T00:00:00+00:00
361
(40, 950)
349
(40, 950)
321
(40, 950)
312
(40, 950)
312
(40, 950)
357
(40, 950)
calculating target year preformed DIC
0
calculating target year preformed DIC
0
calculating target year preformed DIC
0
calculating target year preformed DIC
0
calculating target year preformed DIC
0
calculating target year preformed DIC
0


Process Process-2:
Process Process-4:
Process Process-6:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/tjarniko/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/tjarniko/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/tjarniko/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
Process Process-3:
  File "/home/tjarniko/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-6-5c53b44508cc>", line 43, in func6
    calc_preind_co2_AOU_method(arrow_array[i][0], 2015, 2050, '2_4pt5')
  File "<ipython-input-6-5c53b44508cc>", line 33, in func4
    calc_preind_co2_AOU_method(arrow_array[i][0], 2015, 2050, '2_4pt5')
  File "<ipython-input-4-ef9a3443efb0>", line 87, in calc_preind_co2_AOU_method
    t_

KeyboardInterrupt: 